In [1]:
import pandas as pd
from amb_sdk.sdk import DarwinSdk

In [2]:
ds = DarwinSdk()
ds.set_url('https://amb-demo-api.sparkcognition.com/v1/')

(True, 'https://amb-demo-api.sparkcognition.com/v1/')

In [3]:
#Set your user id and password accordingly
USER='idunlap@rocketmail.com'
PW='5uVGHsTHrQ'

# Set path to datasets - The default below assumes Jupyter was started from amb-sdk/examples/Enterprise/
# Modify accordingly if you wish to use your own data
PATH_TO_DATASET='sets/'
TRAIN_DATASET='18_19_Season_Stats_Per_100_Possessions_SPP.csv'
# A timestamp is used to create a unique name in the event you execute the workflow multiple times or with 
# different datasets.  File names must be unique in Darwin.
import datetime
ts = '{:%Y%m%d%H%M%S}'.format(datetime.datetime.now())

In [4]:
# Import necessary libraries
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image
from time import sleep
import os
import numpy as np
from sklearn.metrics import classification_report


In [ ]:
status, msg = ds.auth_login_user(USER,PW)
if not status:
    print(msg)

In [6]:
# Preview dataset
df = pd.read_csv(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
df.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT_,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,93,Wendell Carter\cartewe01,C,19,CHI,44,44,1110,180,371,...,0.795,87,220,307,78,26,58,65,152,455
1,11,Jarrett Allen\allenja01,C,20,BRK,79,79,2074,335,564,...,0.710,192,467,659,108,43,118,102,182,872
2,17,Ike Anigbogu\anigbik01,C,20,IND,3,0,6,0,3,...,NaN,1,2,3,1,0,1,1,0,0
3,25,Deandre Ayton\aytonde01,C,20,PHO,71,70,2183,509,870,...,0.746,223,506,729,125,61,67,126,209,1159
4,31,Mohamed Bamba\bambamo01,C,20,ORL,47,1,766,117,243,...,0.587,64,169,233,39,13,64,43,102,292


In [7]:
# Upload dataset
status, dataset = ds.upload_dataset(os.path.join(PATH_TO_DATASET, TRAIN_DATASET))
if not status:
    print(dataset)

In [8]:
# clean dataset
target = "Pos"
status, job_id = ds.clean_data(TRAIN_DATASET, target = target)

if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Requested', 'starttime': '2019-04-17T20:53:38.381112', 'endtime': None, 'percent_complete': 0, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['18_19_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['f828a1f6cd61486288bcea8157a2c44b'], 'model_name': None, 'job_error': None}
{'status': 'Complete', 'starttime': '2019-04-17T20:53:38.381112', 'endtime': '2019-04-17T20:53:41.053062', 'percent_complete': 100, 'job_type': 'CleanDataTiny', 'loss': None, 'generations': None, 'dataset_names': ['18_19_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': ['f828a1f6cd61486288bcea8157a2c44b'], 'model_name': None, 'job_error': ''}


In [9]:
model = target + "_model0" + ts
status, job_id = ds.create_model(dataset_names = TRAIN_DATASET, \
                                 model_name =  model, \
                                 max_train_time = '00:02')
if status:
    ds.wait_for_job(job_id['job_name'])
else:
    print(job_id)

{'status': 'Taken', 'starttime': '2019-04-17T20:53:54.597115', 'endtime': None, 'percent_complete': 0, 'job_type': 'TrainModel', 'loss': None, 'generations': 0, 'dataset_names': ['18_19_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'Pos_model020190417205336', 'job_error': None}
{'status': 'Running', 'starttime': '2019-04-17T20:53:54.597115', 'endtime': None, 'percent_complete': 5, 'job_type': 'TrainModel', 'loss': 1.3905984163284302, 'generations': 5, 'dataset_names': ['18_19_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'Pos_model020190417205336', 'job_error': ''}
{'status': 'Running', 'starttime': '2019-04-17T20:53:54.597115', 'endtime': None, 'percent_complete': 5, 'job_type': 'TrainModel', 'loss': 1.3905984163284302, 'generations': 5, 'dataset_names': ['18_19_Season_Stats_Per_100_Possessions_SPP.csv'], 'artifact_names': None, 'model_name': 'Pos_model020190417205336', 'job_error': ''}
{'status': 'Running', 'star

In [ ]:
# Retrieve feature importance of built model
status, artifact = ds.analyze_model(model)
sleep(1)
if status:
    ds.wait_for_job(artifact['job_name'])
else:
    print(artifact)
status, feature_importance = ds.download_artifact(artifact['artifact_name'])

In [ ]:
feature_importance[:10]